In [ ]:
## Given 
## transport cost matrix
## Supply values 
## Demand values
## Finding minimum cost
## Applying MODI method to check optimality
## Handling Degeneracy

In [ ]:
import copy
import numpy as np
from math import isnan
from collections import Counter
### North-west corner method to find Initial Basic Feasible Soln
def nwcr():
  supply_size=int(input("Enter the number of supply centers "))
  demand_size=int(input("Enter the number of demand centers "))
  demand = []
  supply = []
  alloc=[[0]*(demand_size) for i in range(supply_size+1)]
  for i in range(1,supply_size+1):
      supply.append(int(input("Enter the supply for "+str(i)+"th centre: ")))
      
  for j in range(1,demand_size+1):
      demand.append(int(input("Enter the demand for "+str(j)+"th centre: ")))
  import numpy as np
  if np.sum(np.array(supply)) == np.sum(np.array(demand)):
    flag=-1
  elif np.sum(np.array(supply)) > np.sum(np.array(demand)):
    flag= 0       ## Supply more
  else:
    flag= 1        ## Demand more
    
  if flag==0:
    cost=[[0]*(demand_size+2) for i in range(supply_size+1)]
    for i in range(supply_size):
      cost[i][demand_size+1]=supply[i]
    for j in range(demand_size):
      cost[supply_size][j]=demand[j]
    cost[supply_size][demand_size]=abs(np.sum(np.array(supply)) - np.sum(np.array(demand)))
  elif flag==1:
    cost=[[0]*(demand_size+1) for i in range(supply_size+2)]
    for i in range(supply_size):
      cost[i][demand_size]=supply[i]
    for j in range(demand_size):
      cost[supply_size+1][j]=demand[j]
    cost[supply_size][demand_size]=abs(np.sum(np.array(supply)) - np.sum(np.array(demand)))
  else:
    cost=[[0]*(demand_size+1) for i in range(supply_size+1)]
    for i in range(supply_size):
      cost[i][demand_size]=supply[i]
    for j in range(demand_size):
      cost[supply_size][j]=demand[j]
  for i in range(supply_size):
    for j in range(demand_size):
      cost[i][j]=int(input("Enter the cost between demand centre "+str(i)+" and supply centre "+str(j)+": "))

  print("The cost matrix is ")
  print(np.matrix(cost))
  copy_cost=copy.deepcopy(np.array(cost)[:-1,:-1])
  No_of_allocations=0
  Total_cost=0
  supply_size_corr=len(cost)-1
  demand_size_corr=len(cost[0])-1
  i=0
  j=0
  while (i < supply_size_corr and j < demand_size_corr):
    x=min(cost[supply_size_corr][j],cost[i][demand_size_corr])
    #print(x)
    cost[supply_size_corr][j]-=x
    cost[i][demand_size_corr]-=x
    No_of_allocations+=1
    Total_cost+=x*cost[i][j]
    alloc[i][j]=x

    if cost[supply_size_corr][j]<cost[i][demand_size_corr]:
      j+=1
    elif cost[supply_size_corr][j]>cost[i][demand_size_corr]:
      i+=1
    else:
      i+=1
      j+=1
  alloc=alloc[:-1]
  print("The allocation Matrix is ")
  print(np.matrix(alloc))
  print("The minimum cost found is: "+ str(Total_cost))
  return alloc,supply,demand,copy_cost,supply_size,demand_size,No_of_allocations

### Checking For Degeneracy
def check_degeneracy(No_of_allocations,supply_size,demand_size):
  return 0 if No_of_allocations==supply_size+demand_size-1 else 1

## Calculating City block distance between two cells
def dist(k,l):
  l= abs(k[0]-l[0]) + abs(k[1]-l[1])
  return l

## Checking if coordinates are valid
def valid_coord(allocs, i, j):
  if i<0 or j<0 or i>=len(allocs) or i>=len(allocs[0]):
    return False
  else: 
    return True

## Detecting a cycle in the allocation matrix: For valid allica
def detect_cycle(alloc_1_k):
  T=copy.deepcopy(alloc_1_k) 
  T=np.array(T)
  while True:
    _xs, _ys = np.nonzero(T)
    xcount, ycount = Counter(_xs), Counter(_ys)

    for x, count in xcount.items():
        if count <= 1:
            T[x,:] = 0
    for y, count in ycount.items():
        if count <= 1: 
            T[:,y] = 0

    if all(x > 1 for x in xcount.values()) \
            and all(y > 1 for y in ycount.values()):
        break
  #print(np.matrix(T))
  for i in range(supply_size):
    number_=np.count_nonzero(T[i])
    if number_>1:
      return -1
  
  for j in range(demand_size):
    number_=np.count_nonzero(T[:,j])
    if number_>1:
      return -1
  return 1

## Creating Cycles for reallocation Returns full path
def create_cycle(allocs, neg_i, neg_j):
  start=(neg_i, neg_j)
  T = np.array(allocs)
  T[start] = 1  
  while True:
      #print(T)
      _xs, _ys = np.where(np.array(T)!=0)
      xcount, ycount = Counter(_xs), Counter(_ys)

      for x, count in xcount.items():
          if count <= 1:
              T[x,:] = 0
      for y, count in ycount.items():
          if count <= 1: 
              T[:,y] = 0

      if all(x > 1 for x in xcount.values()) and all(y > 1 for y in ycount.values()):
          break

  fringe = set(tuple(p) for p in np.argwhere(T > 0))
  size = len(fringe)

  path = [start]
  while len(path) < size:
    last = path[-1]
    if last in fringe:
        fringe.remove(last)
    next = min(fringe, key=lambda xy : dist(last, (xy[0], xy[1])))
    path.append(next)
  return path
def get_min_in_cycle(allocs, cycle_coordinates,t_d):
    min = 1000000
    for i in range(len(cycle_coordinates)):
      #print(allocs[i[0]][i[1]])
      if t_d[i]==-1 and min > int(allocs[cycle_coordinates[i][0]][cycle_coordinates[i][1]]) and int(allocs[cycle_coordinates[i][0]][cycle_coordinates[i][1]]) > 1:
        #print("hi")
        min = allocs[cycle_coordinates[i][0]][cycle_coordinates[i][1]]
    return min

## Reallocates weights on cycle creations
def modify_cycle(allocs, cycle_coordinates):
    t = 1
    t_d=[1]
    for i in cycle_coordinates:
        x, y = i
        if t == 1:
            t = -1
            t_d.append(t)    
        else:
            t = 1
            t_d.append(t)
    t_d=t_d[:-1]
    #print(min)
    min = get_min_in_cycle(allocs, cycle_coordinates,t_d)
    t=1
    for i in cycle_coordinates:
        x, y = i
        if allocs[x][y]!=1:
          allocs[x][y] = allocs[x][y] + int(t*min)
        else:
          allocs[x][y] = int(t*min)
        if t == 1:
            t = -1
        else:
            t = 1
    
    return np.array(allocs)

## Obtaining allocated cells
def get_all_allocated_coordinates(allocs):
    coordinates = []
    for i in range(len(allocs)):
        for j in range(len(allocs[0])):
            if allocs[i][j] > 0:
                coordinates.append((i, j))
    return coordinates

## Helper for UV finding
def isAnyNan(l):
    for i in l:
        if isnan(i):
            return True
    return False

## Calculating the UV values 
def get_uv(costs, allocs):
    u = []
    v = []
    for i in costs:
        u.append(float('nan'))
        if len(v) == 0:
            for j in costs[0]:
                v.append(float('nan'))
    u[0] = 0
    coordinates = get_all_allocated_coordinates(allocs)
    stopLoopCount = 0
    while isAnyNan(u) or isAnyNan(v):
        stopLoopCount += 1
        for x, y in coordinates:
            if not isnan(u[x]) and isnan(v[y]):
                v[y] = costs[x][y] - u[x]
            elif isnan(u[x]) and not isnan(v[y]):
                u[x] = costs[x][y] - v[y]
        if stopLoopCount % 100 == 0:
            ans = input("Loop ran for {0} times. Do you want to continue? [y] ".format(stopLoopCount))
            if ans != 'y':
                break
    return u, v

### Detecting if the soln is optimal, using dij>=0 or not
## If not optimal returns the index of the most negative 
def isValid(costs, allocs, u, v):
    #print(u, v)
    validity= [[0]*(len(costs[0])) for i in range(len(costs)+1)]
    for i in range(len(costs)):
        for j in range(len(costs[0])):
            if allocs[i][j] == 0:
                validity[i][j] = costs[i][j] - u[i] - v[j]
                #print('#', x, i, j)
    min=np.min(np.array(validity))
    if min<0:
      ind=np.where(validity==min)
      indices=list(zip(ind[0],ind[1]))
      return (indices[0])
    else:
        return -1, -1

## Converting Degenerate solutions into non degenerating: allocating a small value
## The small value is taken 1 due to use of (int) matrix. Not possible to allocate value<1
## rounded to 0
def convert_to_non_degenerate(alloc, cost):
  cost_1=np.array(cost)
  alloc_1=np.array(alloc)
  cost_mod=sorted(list(cost_1.flatten()))
  index=0
  iter=0
  while iter<len(cost_mod):
    flag=0
    ind=np.where(cost_1==cost_mod[index])
    indices=list(zip(ind[0],ind[1]))
    indices=list(set(indices))
    #print(indices)
    for l in range(len(indices)):
      alloc_1=np.array(alloc)
      #print("Before allocation")
      #print(np.matrix(alloc_1))
      if alloc_1[indices[l][0]][indices[l][1]]==0:
        alloc_1[indices[l][0]][indices[l][1]]=1
      else:
         continue
      #print("After allocation")
      #print(np.matrix(alloc_1))
      detect=detect_cycle(alloc_1)
      if detect==1:
        #print("independent allocation found")
        flag=1
        break
      else:
        pass
         #print("independent allocation not found")
    if flag==1:
      return alloc_1
    index+=1
    iter+=1

  return -1

## Caller
alloc,supply,demand,cost,supply_size,demand_size,No_of_allocations=nwcr()
if check_degeneracy(No_of_allocations,supply_size,demand_size)==1:
  print("Degeneracy found")
  alloc_1=convert_to_non_degenerate(alloc, cost)
else:
  alloc_1=copy.deepcopy(alloc)
print("allocation matrix is ")
print(np.matrix(alloc_1))
u, v = get_uv(cost, alloc_1)
x, y = isValid(cost, alloc_1, u, v)
if x==-1 and y==-1:
  print("optimal allocation")
  print(np.matrix(alloc_1))  
else:
  cnt=1
  while x != -1 and y != -1:
        print("Iteration "+str(cnt))
        cnt+=1
        cycle_coordinates = create_cycle(alloc_1, x, y)
        print("closed path found")
        print(cycle_coordinates)
        alloc_1=modify_cycle(alloc_1, cycle_coordinates)
        print("New allocations are")
        print(np.matrix(alloc_1))
        if check_degeneracy(np.count_nonzero(alloc_1),supply_size,demand_size)==1:
          print("Degeneracy found")
          alloc_1=convert_to_non_degenerate(alloc_1, cost)
          print("After removing degeneracy")
          print(np.matrix(alloc_1))
        u, v = get_uv(cost, alloc_1)
        x, y = isValid(cost, alloc_1, u, v)
        if x!=-1 or y!=-1:
          print("optimal allocation not found")
          print(np.matrix(alloc_1)) 
          print("Going for next iteration")
  print("optimal allocation found")
  print(np.matrix(alloc_1)) 
print("cost matrix is ")
print(np.matrix(cost))
print("allocation matrix is ")
print(np.matrix(alloc_1))
Total_cost=0
for i in range(len(cost)):
  for j in range(len(cost[0])):
    if alloc_1[i][j]!=1:
      Total_cost+=cost[i][j]*alloc_1[i][j]
print("The optimal found cost is "+str(Total_cost))

Enter the number of supply centers 4
Enter the number of demand centers 5
Enter the supply for 1th centre: 80
Enter the supply for 2th centre: 60
Enter the supply for 3th centre: 40
Enter the supply for 4th centre: 20
Enter the demand for 1th centre: 60
Enter the demand for 2th centre: 60
Enter the demand for 3th centre: 30
Enter the demand for 4th centre: 40
Enter the demand for 5th centre: 10
Enter the cost between demand centre 0 and supply centre 0: 4
Enter the cost between demand centre 0 and supply centre 1: 3
Enter the cost between demand centre 0 and supply centre 2: 1
Enter the cost between demand centre 0 and supply centre 3: 2
Enter the cost between demand centre 0 and supply centre 4: 6
Enter the cost between demand centre 1 and supply centre 0: 5
Enter the cost between demand centre 1 and supply centre 1: 2
Enter the cost between demand centre 1 and supply centre 2: 3
Enter the cost between demand centre 1 and supply centre 3: 4
Enter the cost between demand centre 1 and s

In [ ]:
21